# SIMPLE TRANSFORMERS TO RATE CRITICS OF MOVIES



In [ ]:
!pip install simpletransformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 91.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.1 MB/s eta

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
#from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
!wget https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
!gunzip criticas_pelis.csv.gz

--2023-11-05 21:36:24--  https://krono.act.uji.es/IDIA/criticas_pelis.csv.gz
Resolving krono.act.uji.es (krono.act.uji.es)... 150.128.97.37
Connecting to krono.act.uji.es (krono.act.uji.es)|150.128.97.37|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4447654 (4.2M) [application/x-gzip]
Saving to: ‘criticas_pelis.csv.gz’

criticas_pelis.csv. 100%[===================>]   4.24M  2.55MB/s    in 1.7s    

2023-11-05 21:36:27 (2.55 MB/s) - ‘criticas_pelis.csv.gz’ saved [4447654/4447654]



In [ ]:
df = pd.read_csv("criticas_pelis.csv", names=["Row", "File", "Text", "Rating"])
df.head()

,Row,File,Text,Rating
0,Row0,File-0,"May, ¿quieres ser mi amigo? May, ¿Quieres ser...",4
1,Row1,File-1,Cómo ponerse en la piel de un kamikaze Es tod...,4
2,Row2,File-10,"Deliciosa comedieta dramática, con tintes rev...",4
3,Row3,File-100,La ironía es el arma de los perdedores y este...,3
4,Row4,File-1000,"Al final, y teniendo en cuenta que esto es el...",3


### We can transform the data to get an equivalent set of labels (classes) than the imported model (sentiment)

In [ ]:
def rating2label(x):
  if x < 3: return 0
  if x > 3: return 2
  return 1

df['labels'] = df['Rating'].apply(rating2label)

df.drop(columns=['Rating'], inplace=True)

df.rename(columns={"Text": "text"})

df.drop(columns=['Row','File'], inplace=True)

In [ ]:
df['labels'].value_counts()/df.shape[0]

2    0.348375
0    0.328520
1    0.323105
Name: labels, dtype: float64

In [ ]:
train_df = df.sample(frac=0.9, random_state=200)
valid_df = train_df.sample(frac=0.1, random_state=200)
test_df  = df.drop(train_df.index)
train_df = train_df.drop(valid_df.index)

In [ ]:
print(train_df.shape, valid_df.shape, test_df.shape)

In [ ]:
display(train_df)

,Text,labels
3691,Película que narra tres historias cuyo nexo e...,0
1325,"Un film pequeño, casi amateur, pero con una c...",1
945,Un experimento que resulta comicamente simple...,0
1759,"Una comedia, una parodia, que sin embargo es ...",2
219,"Entrañable, divertida, original, fresca. Este...",2
...,...,...
646,"Aún asi, siempre se puede decir aquello de me...",1
3351,La esencia de su maestría reside en un argume...,2
3592,"Bien, bastante buena. Una buena película en l...",2
661,Me gustó mucho y creo que si van desprevenido...,2


In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.manual_seed = 42
model_args.overwrite_output_dir = True
model_args.train_batch_size = 32
#model_args.regression = True
#model_args.pos_weight=[list-of-label-weights]
model_args.evaluate_during_training = True


In [ ]:
model = ClassificationModel(
    "roberta",
    "pysentimiento/robertuito-sentiment-analysis",
    args = model_args
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [ ]:
predictions, raw_outputs = model.predict(["esta película es muy mala",\
                                          "vaya porquería, si lo sé no vengo!",\
                                          "una maravilla de argumento"])

predictions

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0, 0, 2])

In [ ]:
# Train the model
model.train_model(train_df, eval_df=valid_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3141 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/99 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/349 [00:00<?, ?it/s]

(99,
 defaultdict(list,
             {'global_step': [99],
              'train_loss': [0.538256824016571],
              'mcc': [0.35543193732364303],
              'eval_loss': [0.8741209506988525]}))

In [ ]:
# Evaluate the model
import sklearn

result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/388 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/49 [00:00<?, ?it/s]

In [ ]:
print(result)

{'mcc': 0.5069633650619425, 'acc': 0.6726804123711341, 'eval_loss': 0.7282478955327248}


In [ ]:
print('Predicciones erróneas:', len(wrong_predictions), 'de', test_df.shape[0])

Predicciones erróneas: 127 de 388


In [ ]:
print(wrong_predictions[0])

{'guid': 0, 'text_a': ' El todo conforma un producto activo y atractivo de ver, con un comienzo prometedor que se va desdibujando con el correr de los minutos. Colateral de Michael Mann desnuda con claridad todos los vicios y virtudes de su realizador. Montaje atractivo rozando lo televisivo, con diferentes y atractivas texturas. Realismo en escenas que impliquen balas (recordar la anterior Heat) y un guión bien construído con cierta tendencia a la acumulación desmedida de diálogos. Mann simepre acierta con la banda de sonido y la música incidental. Los personajes estereotipados (malo/bueno) que con el correr de la cinta y el conocimiento propio, revelan mas similitudes que diferencias.En sus anteriores films estos ingredientes ya estaban incluídos (la serie Miami Vice, Fuego contra fuego, El Informante, Alí), todos ellos en mayor o menor medida films logrados pero en éste se notan más claramente, como si Mann hiciera y quisiese perfeccionar siempre la misma película.El todo conforma u